# LLM4Code **Translation** + HPC 

***LMStudio API*** or ***OLLAMA API*** Jupyter Notebook version

Code translation source and target scripts from:
https://github.com/zjin-lcf/HeCBench/tree/master/src

## Setup

In [3]:
from IPython.display import display, Markdown
import requests
import json
import re
import os
import time
from datetime import datetime
import subprocess
import shlex  # for splitting command line arguments
from openai import OpenAI
import ollama  # !pip install ollama
from prompt_dictionary import PromptDictionary

# Used in similarity metrics
import difflib
import tokenize  # apparently only defined for Python code
import tiktoken
from io import BytesIO

# Enable the autoreload extension
%load_ext autoreload

# Reload all modules imported with %aimport every time before executing subsequent code.
%autoreload 1
    
# We need to reload the prompt dictionary because it is under development
%aimport prompt_dictionary
    
# Text formatting options
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## LLM Configuration Methods

In [5]:
"""
Local hosted LLM via LMStudio.ai API, GGUF quantized models
"""
def lmstudio_llm(model, system_prompt, content_prompt):
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
    
    chat_response = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": content_prompt}
      ],
      temperature=0.2,
      stream=False
    )

    response_text = chat_response.choices[0].message.content

    return response_text

In [6]:
"""
Local hosted LLM via OLLAMA API, GGUF quantized models

Requirement: ollama serve [from server command line]
Requirement: ollama.pull('modelname') [from Python script to pre-load selected model at the server]

ollama.list() [preview what is available to use on the server]

Example: model='codestral:latest'
"""
def ollama_llm(model, num_ctx, system_prompt, content_prompt):
    # Ollama serve must already be running on a port on this server 
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": content_prompt}
        ],
        stream=False,
        options=dict(
            temperature=0.2,
            top_p=0.9,
            num_ctx=num_ctx
        )
    )

    response_text = response['message']['content']

    return response_text

In [7]:
def unload_ollama_model(model):
    print("\nUnloading Ollama server...")
    data_curl = {
        "model": model,
        "keep_alive": 0
    }
    data_curl_json = json.dumps(data_curl)
    curl_command_unload_LLM = [
        'curl',
        'http://localhost:11434/api/generate',
        '-d',
        data_curl_json
    ]
    result_unload_LLM = subprocess.run(curl_command_unload_LLM, capture_output=True, text=True)
    print("\nUnload LLM:\n", result_unload_LLM.stdout)
    print(result_unload_LLM.stderr)

## Pipeline Utility Method Definitions

### LLM Methods

In [8]:
def code_knowledge_llm(llm_source, model, num_ctx, system_prompt, input_context, target_lang, display_response):

    content_prompt = """
    Before translating the code, please provide a summary description the following background knowledge and context on the programming language.
    """

    if target_lang == "OMP":
        content_prompt = content_prompt + """ 
        Specifically, describe how to implement the omp pragma directive 'target teams' for distributing for loops on a GPU.
        """
        
    content_prompt = content_prompt + """ 
    Only respond with your summary description and avoid repeating the code: 
    """

    content_prompt = content_prompt + input_context
    
    if llm_source == "argo":
        response_text = argo_llm(model, system_prompt, content_prompt)
    elif llm_source == "lmstudio":
        response_text = lmstudio_llm(model, system_prompt, content_prompt)
    elif llm_source == "ollama":
        response_text = ollama_llm(model, num_ctx, system_prompt, content_prompt)

    print("\n\n----- " + color.BLUE + "CONTEXT KNOWELEDGE SUMMARY" + color.END + " -----")
    print("(generated by the LLM)\n")
    if display_response:
        display(Markdown(response_text))

    return response_text

In [9]:
def code_description_llm(llm_source, model, num_ctx, system_prompt, input_code, display_response):

    content_prompt = """
    Before translating the code, please provide a summary description of the following code. 
    Only respond with your summary description and avoid repeating the code: 
    """
    
    content_prompt = content_prompt + input_code
    
    if llm_source == "argo":
        response_text = argo_llm(model, system_prompt, content_prompt)
    elif llm_source == "lmstudio":
        response_text = lmstudio_llm(model, system_prompt, content_prompt)
    elif llm_source == "ollama":
        response_text = ollama_llm(model, num_ctx, system_prompt, content_prompt)

    print("\n\n----- " + color.BLUE + "CODE DESCRIPTION" + color.END + " -----")
    print("(generated by the LLM)\n")
    if display_response:
        display(Markdown(response_text))

    return response_text

In [10]:
def code_generator_llm(llm_source, model, num_ctx, system_prompt, content_prompt, display_response):

    if llm_source == "argo":
        response_text = argo_llm(model, system_prompt, content_prompt)
    elif llm_source == "lmstudio":
        response_text = lmstudio_llm(model, system_prompt, content_prompt)
    elif llm_source == "ollama":
        response_text = ollama_llm(model, num_ctx, system_prompt, content_prompt)
        
    print("\n\n----- " + color.BLUE + "GENERATED CODE" + color.END + " -----\n\n")
    if display_response:
        display(Markdown(response_text))

    # Extract only the code block - 
    # Search for the ``` pattern in the text
    #   Regular expression pattern to match text between triple backticks 
    #   that represent code blocks within generated responses from the LLM.
    pattern = r"```(.*?)```"
    match = re.search(pattern, response_text, re.DOTALL)
    # print("RESPONSE TEXT -- ", response_text)
    # print("MATCH -- ", match)
    if match:
        extracted_text = match.group(1)
    else:
        extracted_text = ""
    
    return response_text, extracted_text

### Code Evaluation Methods

In [11]:
def compile_code(source_file, lang, code_compiler, code_compiler_kwds):
       
    # Split the compile keywords into a list of arguments
    compile_kwds_list = shlex.split(code_compiler_kwds)

    if lang == "OMP":
        # The desired name of the compiled program
        output_file = source_file.split('.cpp')[0]
        # Construct the compilation command
        compile_command = [code_compiler] + compile_kwds_list + [source_file, '-o', output_file]
    
    elif lang == "CUDA":
        # The desired name of the compiled program
        output_file = source_file.split('.cu')[0]
        # Construct the compilation command
        compile_command = [code_compiler] + compile_kwds_list + [source_file, '-o', output_file]
        
    print("Compile command: " + str(compile_command))
    
    # Execute the compilation command
    result = subprocess.run(compile_command, capture_output=True, text=True)

    # Check if the compilation was successful
    return_result = ""
    if result.returncode == 0:
        print("\n" + color.BOLD + "Compilation of code successful!" + color.END)
    else:
        # If there was an error, print the error message
        return_result = result.stderr
        print("\n" + color.BOLD + color.RED + "***** Compilation failed." + color.END)
        print(" Error message: ")
        print(return_result)
        
    return return_result, output_file

In [12]:
def execute_code(executable_file, exe_input_parameters):

    # Capture current directory to switch back after code execution
    initial_dir = os.getcwd()

    # Extract the compiled program to execute
    directory_path = os.path.dirname(executable_file)
    executable = os.path.basename(executable_file)

    # Change the working directory to the subdirectory of the executable
    os.chdir(directory_path)

    # Ensure the binary has execute permissions
    subprocess.run(['chmod', '+x', executable])

    # Build execute command
    execute_command = ['./' + executable] + exe_input_parameters
    print("Execute command: " + str(execute_command) + "\n")

    # Execute the binary object with input parameters, if provided
     
    # Ensure the file is open and available before executing
    if os.path.exists(executable):
        try:
            # Open the file to ensure it is accessible
            with open(executable, 'rb') as code_file:
                # Start the process
                process = subprocess.Popen(execute_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                stdout_lines = []
                stderr_lines = []
                exception_error = ""
                
                # Print the output in real-time
                try:
                    # Read stdout in a separate thread or in a non-blocking way
                    while True:
                        line = process.stdout.readline()
                        if not line:
                            break
                        print(line, end='')  # Print each line as it is received
                        stdout_lines.append(line)
                    process.stdout.close()
            
                    # Read stderr in a separate thread or in a non-blocking way
                    while True:
                        line = process.stderr.readline()
                        if not line:
                            break
                        stderr_lines.append(line)
                    process.stderr.close()
                
                    # Wait for the process to complete and get the return code
                    return_code = process.wait()
                    
                    if return_code != 0:
                        error_output = ''.join(stderr_lines)
                        print("Program Execution Error:", error_output)

                except Exception as e:
                    print("An error occurred while executing the program: ", e)
                    exception_error = str(e)
        except IOError as e:
            print(f"Pipeline error | An I/O error occurred: {e.strerror}")
        except Exception as e:
            print(f"Pipeline error | An error occurred: {e}")
    else:
        print(f"Pipeline error | The file {executable} does not exist.")
    
    
    # Check if the program ran successfully
    return_result = "0"  # default OK
    if return_code == 0:
        print("\n" + color.BOLD + "Program executed successfully!" + color.END)
    else:
        # If there was an error during execution, print the error message
        return_result = "Program exited with return code: " + str(return_code) + " "
        if return_code == -11:
            return_result += "Segmentation fault detected."
        stderr_output = ''.join(stderr_lines)
        # stdout_output = ''.join(stdout_lines)
        # if stdout_output:
        #    return_result += "\nStandard Output: " + stdout_output
        if exception_error != "":
            return_result += "\nException Error: " + exception_error
        if stderr_output:
            return_result += "\nStandard Error: " + stderr_output
        print("\n" + color.BOLD + color.RED + "***** Execution failed." + color.END)
        print("Error message: ")
        print(return_result)
    
    os.chdir(initial_dir)

    stdout_output = ''.join(stdout_lines)
    
    return return_result, stdout_output

In [13]:
def token_count(doc_text):
    tokenizer = tiktoken.get_encoding("cl100k_base")

    encoding = tiktoken.Encoding(
        name="gpt-35-turbo",
        pat_str=tokenizer._pat_str,
        mergeable_ranks=tokenizer._mergeable_ranks,
        special_tokens={
            **tokenizer._special_tokens,
            "<|im_start|>": 100264,
            "<|im_end|>": 100265,
        },
    )

    tokens_doc = encoding.encode(
        doc_text, allowed_special={"<|im_start|>", "<|im_end|>"}
    )

    return len(tokens_doc)
    
def tokenize_by_tiktoken(code):
    # Get an encoding (e.g., "cl100k_base")
    enc = tiktoken.get_encoding("cl100k_base")
    
    # Encode and decode a sample text
    encoded_tokens = enc.encode(code)
    decoded_code = enc.decode(encoded_tokens)

    # Verify that the decoding works correctly
    assert decoded_code == code

    return encoded_tokens

def tokenize_by_tokenize(code):
    # Includes an error handling to "skip" over bad token conversions
    tokens = []
    try:
        readline = BytesIO(code.encode('utf-8')).readline
        token_gen = tokenize.tokenize(readline)
        while True:
            try:
                token = next(token_gen)
                tokens.append(token)
            except IndentationError as e:
                print(f"Indentation error in the code: {e}")
                continue
            except StopIteration:
                break
    except Exception as e:
        print(f"Unexpected error: {e}")

    return tokens

def tokenize_code(code, method="tokenize"):

    if method == "tokenize":
        tokens = tokenize_by_tokenize(code)
    elif method == "tiktoken":
        tokens = tokenize_by_tiktoken(code)
    
    return tokens
    
def token_similarity(code1, code2, method):
    tokens1 = tokenize_code(code1, method)
    tokens2 = tokenize_code(code2, method)
    sequence_matcher = difflib.SequenceMatcher(None, tokens1, tokens2)
    return sequence_matcher.ratio()

def find_line_in_lines(line, lines):
    """Helper function to find a line in a list of lines."""
    for i, l in enumerate(lines):
        if line.strip() == l.strip():
            return i
    return -1

def compare_lines_with_reordering(code1, code2):
    """Compare lines of code1 with lines of code2 allowing reordering."""
    lines1 = code1.splitlines()
    lines2 = code2.splitlines()
    
    matched_lines = 0
    for line1 in lines1:
        index = find_line_in_lines(line1, lines2)
        if index != -1:
            matched_lines += 1
            lines2.pop(index)  # Remove the matched line to prevent duplicate matching
    
    total_lines = max(len(lines1), len(code2.splitlines()))
    similarity_ratio = matched_lines / total_lines if total_lines > 0 else 0
    return similarity_ratio

### The Pipeline

In [14]:
def auto_code_llmgeneration_pipeline(this_llm_systemcode_prompt,
                                     this_llm_prompt,
                                     llm_source, 
                                     model,
                                     num_ctx,
                                     # arrayormatrix_size_range,
                                     # thread_count_range,
                                     use_context,
                                     input_context,
                                     input_code,
                                     target_code,
                                     translated_code_extension,
                                     target_lang,
                                     code_compiler,
                                     code_compiler_kwds,
                                     directory,
                                     file_name,
                                     target_file_path,
                                     display_response,
                                     test_with_error,
                                     generate_test_case,
                                     yes_execute,
                                     exe_input_parameters,
                                     meta_data_file
                                    ):
    
    # Test compile and execute the target souurce code to validate compiler and execution environment
    print("\n\n----- " + color.BLUE + "TARGET SOURCE CODE: Test compile and execute" + color.END + " -----")
    print("\nTest compile original target code: " + str(target_file_path))
    compile_result, execute_file = compile_code(target_file_path, target_lang, code_compiler, code_compiler_kwds)
    if compile_result != "":
        return False
    if yes_execute:
        print("\nTest executing the compiled original target code: " + str(execute_file))
        exe_file_path = execute_file  # directory + "/" + execute_file
        execute_result, exe_output_source = execute_code(exe_file_path, exe_input_parameters)
        if execute_result != "0":
            return False
    
    print("\n\n")

    compile_result = "start"
    content = ""
    execute_result = ""
    error_message = ""
    error_source = ""  # "compiled" or "executed"
    code_block = ""
    self_correction_counter = -1

    general_system_prompt = prompts.get_system_prompt("general_system")
    
    # Augmented prompting to handle processing of provided knowledge context
    if use_context:
        content = "[BEGIN CONTEXT KNOWLEDGE] " + input_context + " [END CONTEXT KNOWLEDGE]\n "
    
        llm_code_knowledge_summary_response = code_knowledge_llm(llm_source, model, num_ctx, general_system_prompt, input_context, target_lang, display_response)
        
        content = content + "[BEGIN CONTEXT SUMMARY] " + llm_code_knowledge_summary_response + " [END CONTEXT SUMMARY]\n "
        
        content = content + "Using the above background context with your summarization, "

        llm_code_description_response = code_description_llm(llm_source, model, num_ctx, general_system_prompt, input_code, display_response)
    
        content = content + "Think carefully before developing the following code that you describe as: "

        content = content + llm_code_description_response
    else:
        content = general_system_prompt + "Think carefully before developing the following request: "

    # Save the original context prompt for re-use during error correction.
    original_context_content_prompt = content
    original_prompt_tokens = token_count(original_context_content_prompt)
    
    content = content + " Now, " + this_llm_prompt
    content = content + " " + input_code
    
    start_time = time.time()
    
    # INITIATE LLM CODE GENERATION ITERATIVE PIPELINE
    while compile_result != "":
        self_correction_counter = self_correction_counter + 1
        
        if compile_result == "start":
            # CALL THE LLM
            content = re.sub(' +', ' ', content)
            print("\n\n----- " + color.BOLD + "PROMPT" + color.END + " (initial, token count: " + str(original_prompt_tokens) + "): \n" + content + "\n")
            code_response, code_block = code_generator_llm(llm_source, model, num_ctx, this_llm_systemcode_prompt, content, display_response)
        else:
            # CALL THE LLM WITH ERROR CORRECTION
            print("\n\n----- " + color.BOLD + color.RED + "SELF-CORRECTION: " + str(self_correction_counter) + color.END + " -----")
            if compile_result == "execute_error":
                error_correction_prompt_intro = "\n -- The above code was executed after a successul compile with " + code_compiler + " " + code_compiler_kwds + " and produced the following execution error: "
                error_message = execute_result
            else:  # Must be a compiler error
                # NOTE: Maybe do not provide the original code in the other language when trying to fix code in the target language.
                error_correction_prompt_intro = "\n -- The above code was compiled with " + code_compiler + " " + code_compiler_kwds + " and produced the following compile error: "
                error_message = compile_result
            
            # error_correction_prompt_outro = " Eliminate this specific error by re-factoring the above code with an appropriate fix while also using the following background context: " + original_context_content_prompt
            # error_prompt = code_block + error_correction_prompt_intro + error_message + error_correction_prompt_outro
            # error_correction_prompt_outro = " Eliminate this specific error by re-factoring the above code with an appropriate fix."
            error_correction_prompt_outro = " Re-factor the above code with a fix to eliminate the stated error. "
            error_prompt = code_block + error_correction_prompt_intro + error_message + error_correction_prompt_outro
            error_prompt_tokens = token_count(error_prompt)
            print("\n\n----- " + color.BOLD + color.RED + "ERROR SELF-CORRECTION" + color.END + " -----")
            print("\nPROMPT (error correction, token count: " + str(error_prompt_tokens) + "): " + error_prompt + "\n")
            error_prompt = re.sub('\n', '', error_prompt)
            code_response, code_block = code_generator_llm(llm_source, model, num_ctx, this_llm_systemcode_prompt, error_prompt, display_response)

        if code_block.startswith('c++') or code_block.startswith('cpp'):
            code_block = code_block[3:]
        if code_block.startswith('c'):
            code_block = code_block[1:]    
    
        # SAVE THE GENERATIVE CODE -- before compiling
        file_path = directory + "/" + file_name + "." + translated_code_extension
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w') as file:
            file.write(code_block)
        file.close()
    
        # Compile the generated code
        compile_result, execute_file = compile_code(file_path, target_lang, code_compiler, code_compiler_kwds)

        print("\nExecute file: " + str(execute_file))

        if yes_execute and compile_result == "" and self_correction_counter <= 7:
            # Unload any previously loaded LLMs in the Ollama server 
            # (clear out the memory)
            if llm_source == "ollama":
                unload_ollama_model(model)
            
            exe_file_path = execute_file  # directory + "/" + execute_file
            execute_result, exe_output = execute_code(exe_file_path, exe_input_parameters,)
            if execute_result != "0":
                compile_result = "execute_error"  # just a flag
                
    # end of while pipeline loop

    end_time = time.time() - start_time

    now = datetime.now()
    dt_string = now.strftime("%m-%d-%Y_t%H_%M_%S")
    
    print("\n-----" + color.GREEN + "CODE GENERATION COMPLETE" + color.END + "-----\n")
    end_time_minutes = round(end_time/60, 2)
    print("CODE GENERATION TIME: " + str(end_time_minutes) + " minutes.\n")
    print("Self-correction count: " + str(self_correction_counter) + "\n")

    # MEASURE SIMILARITY BETWEEN GENERATED TRANSLATION CODE AND ORIGINAL TARGET CODE
    similarity_tokens = token_similarity(target_code, code_block, "tokenize")
    print(f"Token similarity (tokenize): {similarity_tokens:.2f}")

    similarity_tokens_tiktoken = token_similarity(target_code, code_block, "tiktoken")
    print(f"Token similarity (tiktoken): {similarity_tokens_tiktoken:.2f}")
    
    similarity_lines = compare_lines_with_reordering(target_code, code_block)
    print(f"Line-based similarity with reordering: {similarity_lines:.2f}")

    meta_data_file += "\n\n"
    meta_data_file += "----- METRICS -----"
    meta_data_file += "\n"
    meta_data_file += "Experiment date-time: " + str(dt_string)
    meta_data_file += "\n"
    meta_data_file += "Time (sec): " + str(round(end_time, 2))
    meta_data_file += "\n"
    meta_data_file += f"Token similarity (tokenize): {similarity_tokens:.2f}"
    meta_data_file += "\n"
    meta_data_file += f"Token similarity (tiktoken): {similarity_tokens_tiktoken:.2f}"
    meta_data_file += "\n"
    meta_data_file += f"Line-based similarity with reordering: {similarity_lines:.2f}"
    meta_data_file += "\n"
    meta_data_file += "Self-correction iteration count: " + str(self_correction_counter)
    if yes_execute:
        meta_data_file += "\n\n\n----- SOURCE: EXECUTION OUTPUT -----"
        meta_data_file += exe_output_source
        meta_data_file += "\n\n\n----- GENERATED: EXECUTION OUTPUT -----"
        meta_data_file += exe_output

    # Write meta data for this experiment
    filename = directory + "/" + "metadata" + "_" + file_name + "__" + str(dt_string) + ".txt"
    with open(filename, 'w') as file:
        file.write(meta_data_file)
    file.close()

    print("Metadata file: " + str(filename))

    return True

## Code Generation Experiment - Setup + Run

_____
***STEP 1/2: Input experimental configuration parameters in the following method.***

In [15]:
def experimental_setup():
    """
    [1] ENTER APPLICATION FOLDER NAME FROM HeCBench
        >> Execution input parameters provided.

    -- matrix-rotate ["10000", "1"]
    -- jacobi [""] (none required)
    -- DNU -- mallocFree ["10000"]
    -- layout ["1"]
    -- atomicCost ["1"]
    -- dense-embedding ["10000", "8", "1"]
    -- pathfinder ["10000", "1000", "1000"]
    -- bsearch ["10000", "1"]
    -- entropy ["10000", "1024", "1"]
    -- colorwheel ["10000", "8", "1"]
    -- randomAccess ["1"]
    """
    
    app_folder_name = "randomAccess"

    
    """
    [2] SELECT THE CODE TRANSLATION MAPPING DIRECTION EXPERIMENT CONFIGURATION
    
    CUDA -- OMP -- HIP -- SYCL -- FORTRAN
    """   
    source_lang = "CUDA"
    target_lang = "OMP"

    # TOGGLE USE CODE KNOWLEDGE CONTEXT FOR TARGET
    use_context = True  # True or False

    # ASSIGN CODE COMPILE COMMAND
    if target_lang == "CUDA":
        code_compiler = "nvcc"
        code_compiler_kwds = "-std=c++14 -Xcompiler -Wall -arch=sm_80 -O3"
        # -arch=sm_70 for aiops01 GPU (Argonne)  # -arch=sm_80 for A100s
        context_knowledge_selection = "cuda"
    elif target_lang == "OMP":
        # Linux_x86_64 or Linux_aarch64
        code_compiler = "/opt/nvidia/hpc_sdk/Linux_aarch64/2024/compilers/bin/nvc++"  # "g++"
        code_compiler_kwds = "-Wall -O3 -Minfo -mp=gpu -gpu=cc80"  # "-fopenmp -O3 -c"
        context_knowledge_selection = "omp"

    # TOGGLE EXECUTE PIPELINE
    yes_execute = True  # True / False
    if yes_execute:
        # Enter execution input parameters, if needed
        # A list of parameter string values, e.g., ["10000", "1"]
        exe_input_parameters = ["1"]
    else:
        exe_input_parameters = [""]

    
    """
    [3] SELECT THE LLM TO USE FOR THE CODE TRANSLATION EXPERIMENT
    """
    # SELECT A MODEL
    
    # -- FROM LMSTUDIO
    # model = "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/"
    # model = "TheBloke/CodeLlama-13B-Instruct-GGUF"
    # model = "lmstudio-community/starcoder2-15b-instruct-v0.1-GGUF" - couldn;t generate anything
    # model = "lmstudio-community/stable-code-instruct-3b-GGUF" - couldn't compile, but 'worked'
    # model = "microsoft/Phi-3-mini-4k-instruct-gguf" - generated garbage, and slow
    # model = "lmstudio-community/codegemma-1.1-7b-it-GGUF" - generated code, faster, but not compiled -- more tries
    # model = "lmstudio-community/Codestral-22B-v0.1-GGUF"  # - nice large context size 32768 -- load it on aiop01 at 28 gpu layers

    # -- FROM OLLAMA
    # model = "codestral:22b-v0.1-q8_0"  # (max token = 32768)
    model = "wizardcoder:33b-v1.1-q8_0"  # (max token = 16384)
    # model = "deepseek-coder-v2:16b-lite-instruct-fp16"  # (max token = 128000)
    # model = "deepseek-coder-v2:16b-lite-instruct-q8_0"
  
    # Set maximum context token count size for selected LLM:
    num_ctx = 16384

    # ASSIGN LLM API SOURCE:
    if "GGUF" in model:
        llm_source = "lmstudio"
    else:
        llm_source = "ollama"

    """
    [4] SET ADDITIONAL PIPELINE PARAMETERS
    """
    experiment = "01a_withcontext"  # Assign any label for experiment name.
    display_response = True
    test_with_error = False
    generate_test_case = False

    # Unload any previously loaded LLMs in the Ollama server 
    # (clear out the memory)
    if llm_source == "ollama":
        unload_ollama_model(model)

    return (app_folder_name, source_lang, target_lang, model, num_ctx, llm_source, use_context,
            context_knowledge_selection, code_compiler, code_compiler_kwds, yes_execute, exe_input_parameters,
            experiment, display_response, test_with_error, generate_test_case)

_____
***STEP 2/2: Run the following block to implement the pipeline with the configuration parameters.***

In [ ]:
# Load prompt dictionary (to capture recent manual changes to script, if any)
from prompt_dictionary import PromptDictionary
prompts = PromptDictionary()

# Store the current home working directory (needed for code execution)
original_home_directory = os.getcwd()


# COLLECT THE EXPERIMENT CONFIGURATIONS
(
    app_folder_name, source_lang, target_lang, model, num_ctx,
    llm_source, use_context, context_knowledge_selection, code_compiler,
    code_compiler_kwds, yes_execute, exe_input_parameters, experiment,
    display_response, test_with_error, generate_test_case
) = experimental_setup()


# IMPORT CONTEXT KNOWLEDGE FOR TARGET PROGRAMMING LANGUAGE
input_context = prompts.get_codeknowledge(str(context_knowledge_selection))


# DEFINE PROMPT TEMPLATES
system_prompt_selection_index = source_lang + "_to_" + target_lang  # Select which prompt to use from the prompt dictionary
this_llm_system_prompt = prompts.get_system_prompt(str(system_prompt_selection_index))

codetranslate_prompt_selection_index = source_lang + "_to_" + target_lang  # Select which prompt to use from the prompt dictionary
this_llm_translatecode_prompt = prompts.get_codetranslate_prompt(str(codetranslate_prompt_selection_index))


# LOAD INPUT CODE TO TRANSLATE
input_code_folder = "HeCBench/" + app_folder_name + "/"
input_code_filename = app_folder_name + "-" + source_lang.lower() + "_main"
if source_lang == "OMP" or source_lang == "SYCL":
    input_code_extension = "cpp"
elif source_lang == "CUDA" or source_lang == "HIP":
    input_code_extension = "cu"
source_file_directory = "translated_code/input_codes/" + input_code_folder
source_file_path = source_file_directory + input_code_filename + "." + input_code_extension
with open(source_file_path, 'r') as file:
    input_code = file.read()
file.close()


# LOAD TARGET CODE TO COMPARE TRANSLATION
target_code_filename = app_folder_name + "-" + target_lang.lower() + "_main"
if target_lang == "OMP" or target_lang == "SYCL":
    translated_code_extension = "cpp"
elif target_lang == "CUDA" or target_lang == "HIP":
    translated_code_extension = "cu"
target_file_directory = "translated_code/input_codes/" + input_code_folder
target_file_path = target_file_directory + target_code_filename + "." + translated_code_extension
with open(target_file_path, 'r') as file:
    target_code = file.read()
file.close()


# SPECIFY OUTPUT CODE SAVE
directory = "translated_code/" + llm_source + "_" + model.replace('/', '--') + "/promptset_" + str(codetranslate_prompt_selection_index)
file_name = input_code_filename + "_translatecode_" + target_lang + "_exp" + experiment


# CREATE METADATA RECORD CONTENT
meta_data_file = "***** EXPERIMENT CONFIGURATION SUMMARY *****\n"
meta_data_file += "\nApplication: " + app_folder_name
meta_data_file += "\n\nTranslation: " + system_prompt_selection_index
meta_data_file += "\nTranslate from: " + input_code_extension
meta_data_file += "\nTranslate to: " + translated_code_extension
meta_data_file += "\n\nLLM Source: " + llm_source
meta_data_file += "\nModel: " + model
meta_data_file += "\nSystem Prompt Index: " + str(codetranslate_prompt_selection_index)
meta_data_file += "\nCode file template: " + directory + "/" + file_name
meta_data_file += "\n\nLLM System Prompt: " + "\n\n" + this_llm_system_prompt
meta_data_file += "\n\nLLM Prompt: " + "\n\n" + this_llm_translatecode_prompt

print("\n" + meta_data_file + "\n")


print("\n\n" + color.BOLD + "***** BEGIN PIPELINE*****" + color.END + "\n\n")

# RUN PIPELINE
complete = auto_code_llmgeneration_pipeline(
    this_llm_system_prompt,
    this_llm_translatecode_prompt,
    llm_source, 
    model,
    num_ctx,
    # arrayormatrix_size_range,
    # thread_count_range,
    use_context,
    input_context,
    input_code,
    target_code,
    translated_code_extension,
    target_lang,
    code_compiler,
    code_compiler_kwds,
    directory,
    file_name,
    target_file_path,
    display_response,
    test_with_error,
    generate_test_case,
    yes_execute,
    exe_input_parameters,
    meta_data_file
)

# Reset to original working directory
os.chdir(original_home_directory)

if complete:
    print("\n\n" + color.GREEN + color.BOLD + "***** PIPELINE COMPLETED *****\n\n" + color.END)
else:
    print("\n\n" + color.RED + color.BOLD + "***** PIPELINE FAILED TO COMPLETE *****\n\n" + color.END)